# Build the logbook for SCAN data on new Spectractor

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- creation date : March 24th 2022


Works directory per directory (day by day)

CCIN2P3:

- works with kernels **anaconda3_auxtel** (with libradtran) at CC
- works with kernel **python 3** locally

In [1]:
import sys
sys.path

['/pbs/throng/lsst/users/dagoret/desc/AuxTelComm/notebookccdm_gen3/runspectractor_standalone/2022_03',
 '',
 '/opt/conda/lib/python3.8/site-packages',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python38.zip',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/lib-dynload',
 '/pbs/home/d/dagoret/.local/lib/python3.8/site-packages',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/Deprecated-1.2.11-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/docopt-0.6.2-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/mimeparse-0.1.3-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/Spectractor-2.1-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/rail-0.1.dev0-

In [2]:
DATE = '20220316'
version="v5.0"  # new orientation

In [3]:
# reconstructed / good /xpos /ypos

if version == "v5.0":
    info={\

          # with red filter
          'exposure_2022031600351_postisrccd.fits':(1,1,300.,1700),\
         }
        
        
        
        
        
        
 

# Imports

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re

plt.rcParams["figure.figsize"] = (16,4)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [5]:
from astropy.io import fits

In [6]:
import imp
MODULE_EXTENSIONS = ('.py', '.pyc', '.pyo')

In [7]:
def package_contents(package_name):
    file, pathname, description = imp.find_module(package_name)
    if file:
        raise ImportError('Not a package: %r', package_name)
    # Use a set because some may be both source and compiled.
    return set([os.path.splitext(module)[0]
        for module in os.listdir(pathname)
        if module.endswith(MODULE_EXTENSIONS)])

In [8]:
package_contents("spectractor")

{'__init__',
 '_version',
 'astrometry',
 'config',
 'logbook',
 'parameters',
 'tools'}

In [9]:
package_contents("spectractor/extractor")

{'__init__',
 'background',
 'chromaticpsf',
 'dispersers',
 'extractor',
 'image',
 'images',
 'psf',
 'spectroscopy',
 'spectrum',
 'targets'}

# Initialisation

In [10]:
dir_logbooks="logbooks"

In [11]:
import os
if not os.path.exists(dir_logbooks):
    os.makedirs(dir_logbooks)

## Get inputs files and sort them according date and number

In [12]:
FLAG_HOLO=True
home=os.getenv("HOME")

In [13]:
if FLAG_HOLO:
    image_path="holo"
    logbook_csv=os.path.join(dir_logbooks,"auxtelholologbook_" + DATE +"_" + version + ".csv")
    logbook_xlsx=os.path.join(dir_logbooks,"auxtelholologbook_" + DATE +"_" + version + ".xlsx")
else:
    image_path="ronchi170lpmm"
    logbook_csv=os.path.join(dir_logbooks,"auxtelronchilogbook_"+ DATE +"_" + version + ".csv")
    logbook_xlsx=os.path.join(dir_logbooks,"auxtelronchilogbook_"+ DATE +"_" + version + ".xlsx")

# Path

In [14]:
HOSTCC=True

In [15]:
# Set path depending on which computer running (according HOSTCC)
if HOSTCC:
    path_auxtel="/sps/lsst/groups/auxtel"
    path_spectractor=os.path.join(path_auxtel,"softs/github/desc/Spectractor")
    path_spectractor_config=os.path.join(path_spectractor,"config")
    path_images=os.path.join(path_auxtel,"data/2022/holo/"+DATE)
    path_output_spectractor=os.path.join(path_auxtel,"data/2022/OutputSpectractor/holo"+DATE)
    
else:
    path_auxtel="/Users/dagoret/DATA/AuxTelData2021"
    path_spectractor=os.path.join(path_auxtel,"/users/dagoret/MacOSX/github/LSST/Spectractor")
    path_spectractor_config=os.path.join(path_spectractor,"config")
    path_images=os.path.join(path_auxtel,"holo/quickLookExp_v2/"+DATE)
    path_output_spectractor=os.path.join(path_auxtel,"holo/OutputSpectractor/"+DATE)

In [16]:
dir_images=path_images

In [17]:
all_files=os.listdir(dir_images)

In [18]:
all_files

['exposure_2022031600387_postisrccd.fits',
 'exposure_2022031600434_postisrccd.fits',
 'exposure_2022031600394_postisrccd.fits',
 'exposure_2022031600358_postisrccd.fits',
 'exposure_2022031600359_postisrccd.fits',
 'exposure_2022031600331_postisrccd.fits',
 'exposure_2022031600386_postisrccd.fits',
 'exposure_2022031600380_postisrccd.fits',
 'exposure_2022031600345_postisrccd.fits',
 'exposure_2022031600427_postisrccd.fits',
 'exposure_2022031600344_postisrccd.fits',
 'exposure_2022031600366_postisrccd.fits',
 'exposure_2022031600379_postisrccd.fits',
 'exposure_2022031600338_postisrccd.fits',
 'exposure_2022031600435_postisrccd.fits',
 'exposure_2022031600373_postisrccd.fits',
 'exposure_2022031600400_postisrccd.fits',
 'exposure_2022031600393_postisrccd.fits',
 'exposure_2022031600337_postisrccd.fits',
 'exposure_2022031600401_postisrccd.fits',
 'exposure_2022031600372_postisrccd.fits',
 'exposure_2022031600365_postisrccd.fits',
 'exposure_2022031600428_postisrccd.fits',
 'exposure_

In [19]:
all_selected_files = []
for filename in all_files:
    if re.search(".fits$",filename):
        all_selected_files.append(filename)
N=len(all_selected_files)

In [20]:
all_selected_files=np.array(all_selected_files)

### Sort files

In [21]:
def file_tag_forsorting(filename):
    #m=re.findall('^Cor_holo4_003_.*([0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]_.*)[.]fits$',filename)
    m=re.findall('exposure.*_(.*)_postisrccd[.]fits$',filename)
    
    return m[0]

In [22]:
file_tag_forsorting('exposure_2022031700310_postisrccd.fits')

'2022031700310'

In [23]:
#all_tags = [ file_tag_forsorting(filename)[0] for filename in all_selected_files]
all_tags = [ file_tag_forsorting(filename) for filename in all_selected_files]
all_tags=np.array(all_tags)
all_tags

array(['2022031600387', '2022031600434', '2022031600394', '2022031600358',
       '2022031600359', '2022031600331', '2022031600386', '2022031600380',
       '2022031600345', '2022031600427', '2022031600344', '2022031600366',
       '2022031600379', '2022031600338', '2022031600435', '2022031600373',
       '2022031600400', '2022031600393', '2022031600337', '2022031600401',
       '2022031600372', '2022031600365', '2022031600428', '2022031600351',
       '2022031600352', '2022031600421', '2022031600420', '2022031600330'],
      dtype='<U13')

In [24]:
indexes=np.argsort(all_tags)

In [25]:
sorted_files=all_selected_files[indexes]

In [26]:
sorted_files

array(['exposure_2022031600330_postisrccd.fits',
       'exposure_2022031600331_postisrccd.fits',
       'exposure_2022031600337_postisrccd.fits',
       'exposure_2022031600338_postisrccd.fits',
       'exposure_2022031600344_postisrccd.fits',
       'exposure_2022031600345_postisrccd.fits',
       'exposure_2022031600351_postisrccd.fits',
       'exposure_2022031600352_postisrccd.fits',
       'exposure_2022031600358_postisrccd.fits',
       'exposure_2022031600359_postisrccd.fits',
       'exposure_2022031600365_postisrccd.fits',
       'exposure_2022031600366_postisrccd.fits',
       'exposure_2022031600372_postisrccd.fits',
       'exposure_2022031600373_postisrccd.fits',
       'exposure_2022031600379_postisrccd.fits',
       'exposure_2022031600380_postisrccd.fits',
       'exposure_2022031600386_postisrccd.fits',
       'exposure_2022031600387_postisrccd.fits',
       'exposure_2022031600393_postisrccd.fits',
       'exposure_2022031600394_postisrccd.fits',
       'exposure_202

In [27]:
N=len(sorted_files)

# Build the logbook from the headers

In [28]:
idx=0
#ccdir='/sps/lsst/groups/auxtel/data/2021/holo/FlipCleans'
ccdir=dir_images
all_dates=[]
all_files=[]
all_seqnum=[]
all_subdir=[]
all_indexes=[]
all_targets=[]
all_filters=[]
all_disperser=[]
all_airmass=[]
all_exposures=[]
all_focus=[]
all_seeing=[]
all_P=[]
all_T=[]
all_H=[]
all_W=[]
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header 
    all_indexes.append(idx)
    all_seqnum.append(hdr0['SEQNUM'])
    all_files.append(filename)
    all_subdir.append(ccdir)
    targetname=hdr0['OBJECT']
    all_targets.append(targetname)
    all_dates.append(hdr0['DATE-OBS'])
    all_airmass.append(hdr0['AMSTART'])
    all_filters.append(hdr0['FILTER'])
    all_disperser.append(hdr0['GRATING'])
    all_exposures.append(hdr0['EXPTIME'])
    all_focus.append(hdr0['FOCUSZ'])
    all_seeing.append(hdr0['SEEING'])
    all_P.append(hdr0['PRESSURE'])
    all_T.append(hdr0['AIRTEMP'])
    all_H.append(hdr0['HUMIDITY'])
    all_W.append(hdr0['WINDSPD'])
       
    
    idx+=1
    

# Make pandas dataframe

## Create pandas dataframe

In [29]:
df=pd.DataFrame()

In [30]:
len(all_airmass)

28

In [31]:
N=len(all_indexes)

## Fill with container info

In [32]:
df['index']=all_indexes
df['date']=all_dates
df['path']=all_subdir
df['file']=all_files
df['num']=all_seqnum
df['object']=all_targets
df['filter']=all_filters
df['disperser']=all_disperser
df['airmass']=all_airmass
df['exposure']=all_exposures
df['focus']=all_focus
df['seeing']=all_seeing
df['pressure']=all_P
df['temperature']=all_T
df['humidity']=all_H
df['windspd']=all_W

## Add dictionnary info

### Initialise new columns

In [33]:
Obj_posXpix=np.full(N,np.nan)
Obj_posYpix=np.full(N,np.nan)
Dx=np.full(N,np.nan)
Dy=np.full(N,np.nan)
run=np.full(N,np.nan)
quality=np.full(N,np.nan)
skip=np.full(N,"    ")

### Fill dictionnary info

In [34]:
# loop on rows index
for idx in np.arange(N):
    filename=all_files[idx]
    # check if filename is in dictionnary
    if filename in info.keys():
        (run_flag,quality_flag,xpos,ypos)=info[filename]
        Obj_posXpix[idx]=int(xpos)
        Obj_posYpix[idx]=int(ypos)
        run[idx]=run_flag
        if quality_flag==1:
            skip[idx] = "    "
            quality[idx]=1
        else:
            skip[idx] = "skip"
            quality[idx]=0

In [35]:
df['Obj-posXpix']=Obj_posXpix
df['Obj-posYpix']=Obj_posYpix
df['Dx']=Dx
df['Dy']=Dy
df['skip']=skip
df['run']=run
df['quality']=quality

In [36]:
pd.set_option('display.max_rows', None)

In [37]:
df

,index,date,path,file,num,object,filter,disperser,airmass,exposure,...,temperature,humidity,windspd,Obj-posXpix,Obj-posYpix,Dx,Dy,skip,run,quality
0,0,2022-03-17T01:59:37.776,/sps/lsst/groups/auxtel/data/2022/holo/20220316,exposure_2022031600330_postisrccd.fits,330,ETA DOR,empty_1,holo4_003,1.385409,20.0,...,10.6,44.0,7.2,NaN,NaN,NaN,NaN,,NaN,NaN
1,1,2022-03-17T02:00:10.693,/sps/lsst/groups/auxtel/data/2022/holo/20220316,exposure_2022031600331_postisrccd.fits,331,ETA DOR,empty_1,holo4_003,1.386483,20.0,...,10.6,44.0,7.4,NaN,NaN,NaN,NaN,,NaN,NaN
2,2,2022-03-17T02:03:04.275,/sps/lsst/groups/auxtel/data/2022/holo/20220316,exposure_2022031600337_postisrccd.fits,337,ETA DOR,empty_1,holo4_003,1.392269,20.0,...,10.6,43.0,7.4,NaN,NaN,NaN,NaN,,NaN,NaN
3,3,2022-03-17T02:03:37.365,/sps/lsst/groups/auxtel/data/2022/holo/20220316,exposure_2022031600338_postisrccd.fits,338,ETA DOR,empty_1,holo4_003,1.393378,20.0,...,10.6,43.0,7.4,NaN,NaN,NaN,NaN,,NaN,NaN
4,4,2022-03-17T02:06:31.026,/sps/lsst/groups/auxtel/data/2022/holo/20220316,exposure_2022031600344_postisrccd.fits,344,ETA DOR,empty_1,holo4_003,1.399309,20.0,...,10.5,43.0,8.2,NaN,NaN,NaN,NaN,,NaN,NaN
5,5,2022-03-17T02:07:03.927,/sps/lsst/groups/auxtel/data/2022/holo/20220316,exposure_2022031600345_postisrccd.fits,345,ETA DOR,empty_1,holo4_003,1.400449,20.0,...,10.5,43.0,7.6,NaN,NaN,NaN,NaN,,NaN,NaN
6,6,2022-03-17T02:11:00.681,/sps/lsst/groups/auxtel/data/2022/holo/20220316,exposure_2022031600351_postisrccd.fits,351,HD111980,empty_1,holo4_003,1.647734,20.0,...,10.5,43.0,13.1,300.0,1700.0,NaN,NaN,,1.0,1.0
7,7,2022-03-17T02:11:33.581,/sps/lsst/groups/auxtel/data/2022/holo/20220316,exposure_2022031600352_postisrccd.fits,352,HD111980,empty_1,holo4_003,1.643280,20.0,...,10.5,43.0,13.1,NaN,NaN,NaN,NaN,,NaN,NaN
8,8,2022-03-17T02:14:30.261,/sps/lsst/groups/auxtel/data/2022/holo/20220316,exposure_2022031600358_postisrccd.fits,358,HD111980,empty_1,holo4_003,1.619945,20.0,...,10.5,43.0,12.1,NaN,NaN,NaN,NaN,,NaN,NaN
9,9,2022-03-17T02:15:03.181,/sps/lsst/groups/auxtel/data/2022/holo/20220316,exposure_2022031600359_postisrccd.fits,359,HD111980,empty_1,holo4_003,1.615702,20.0,...,10.5,44.0,6.9,NaN,NaN,NaN,NaN,,NaN,NaN


# Make logbook


In [38]:
df.to_excel(logbook_xlsx)
df.to_csv(logbook_csv)

In [39]:
logbook_csv

'logbooks/auxtelholologbook_20220316_v5.0.csv'